***Introduction/Background***

Denver, CO is one of the fastest growing cities in the United States, which makes it a potentially fruitful location for new restaurants. Given its size, however, there are numerous potential locations for a new restaurant. One way to gain insight about where to place a new restaurant is by examining where in the city other new restaurants are choosing to locate. Fortunately, Denver makes available a list of pending business applications and the addresses in the city associated with these applications. 

Knowing where new restaurants are choosing to locate within the city is useful information, but we may not want to locate a new restaurant nearby ourselves because of increased competition. Instead, we can characterize the surrounding neighborhood of these upcoming restaurants and use these characteristics to find other areas in the city for potential new restaurant locations.

The aim of this study is to generate several sets of neighborhood characteristics being sought by new restaurants in Denver using Foursquare data. This study would be interesting to several different audiences including restrauteurs, local chambers of commerce city planners, and urban developers looking either to start a new restaurant, promote neighborhoods suitable for new restaurants, or create neighborhoods that are desirable to restrauters. 


***Data***

Denver, CO provides data on pending business applications within the city publicly available at https://www.denvergov.org/opendata/dataset/city-and-county-of-denver-pending-business-licenses.

Within this dataset, the city provides the type of license being applied for and the address associated with the business license application. Filtering by license type and sub-license type for 'Retail Food Establishment' and 'Restaurant', respectively, it is possible to generate a list of addresses of restaurant businesses with pending applications. After generating coordinate data for the addresses, we can query Foursquare for surrounding venues and perform clustering to generate sets of neighborhood characteristics associated with new restaurant business applications in Denver.


In [50]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [51]:
df = pd.read_csv('pending_business_licenses.csv')
df.head()

,BFN,LICENSE_TYPE,LICENSE_SUB_TYPE,LICENSE_STATUS,EXPIRATION_DATE,ENTITY_NAME,ESTABLISHMENT_ADDRESS
0,2021-BFN-0002021,Retail Food Establishment,Mobile,Pending,4/6/2021 12:26:16 PM,AISHA'S KITCHEN LLC,496 FLORENCE ST AURORA 80010
1,2021-BFN-0002028,Retail Food Establishment,Restaurant,Pending,4/6/2021 1:23:48 PM,PETUNIA BAKE SHOP LLC,3000 W 23rd Ave Denver 80211
2,2019-BFN-0001302,Combined License,NaN,Pending,2/14/2019 12:54:33 PM,PROST BREWING COMPANY LLC,2540 19th St Denver 80211
3,2020-BFN-0000952,Swimming Pool,NaN,Pending,2/12/2020 12:22:54 PM,CONCHA TOWNHOUSE ASSOCIATION,7882 E Iowa Ave Denver 80231
4,2019-BFN-0011393,Retail Food Establishment,Restaurant,Pending,12/9/2019 8:20:12 AM,RAISING CANES RESTAURANTS LLC,1190 S Colorado Blvd Denver 80246


***Methods***

After obtaining a list of all pending business licenses in the city of Denver, a quick count revealed 690 pending applications. Out of these 690 applications, 182 were categorized as 'Retail Food Establishment'. Within this license type, there were 4 sub-types: 'Mobile', 'Restaurant', 'Restaurant - Stand Up', and 'Restaurant - Temporary'. Unfortunately, the documentation of the data set did not provide any additional information describing the nature of these sub-types. Given time constraints, I decided to limit the analysis to the 'Restaurant' sub-type, which resulted in 83 applications. Further review of the remaining applications revealed that the list included expired licenses. Due to the rapidly changing nature of growing cities like Denver, I decided to limit to analysis to pending applications that are either active or expired within 2020. This resulted in only removing 1 application leaving a total of 82 applications for further analysis.

To obtain coordinates for the application locations, I separated the provided addresses into the street address, city, state, and zip code and created a CSV file suitable for batch processing by the US Census Geocoder per their guidelines (https://geocoding.geo.census.gov/geocoder/locations/addressbatch?form). I obtained coordinates for 68 of the applications from the US Census Geocoder. The remainder I manually looked up on Google Maps.

The Foursquare API was then queried using the coordinates of each application to obtain venue data within 500 meters of each address. 500 meters was chosen as neighborhoods in Denver appear to be spaced roughly a kilometer apart, thus a radius of 500 meters around the address would be analagous to the neighborhood of the address. Each venue resulting from the query was then binned into their respective categories provided by Foursquare. The category data was transformed using one-hot encoding and the means of each category per application was calculated. K-means clustering was then applied to the matrix of the ten most frequent venues per address. A k of 5 chosen to obtain 5 sets of neighborhood characteristics of pending restaurant business applications. The labels generated via k-means were used to generate a map of the application addresses and to assess differences in characteristics of nearby addresses labelled differently.

In [52]:
df.shape

(690, 7)

In [53]:
df['EXPIRATION_DATE'] = pd.to_datetime(df['EXPIRATION_DATE'])

In [54]:
df.groupby('LICENSE_TYPE').count()

,BFN,LICENSE_SUB_TYPE,LICENSE_STATUS,EXPIRATION_DATE,ENTITY_NAME,ESTABLISHMENT_ADDRESS
LICENSE_TYPE,,,,,,
Auto Parts Recycler,1,0,1,1,1,1
Boarding Home,5,0,5,5,5,5
Boarding Home - Personal Care,2,0,2,2,2,2
Body Art Est - Permanent,7,0,7,7,7,7
Burglar Alarm System Company,3,0,3,3,3,3
Child Care,33,33,33,33,32,33
Child Care Center,3,0,3,3,3,3
Child Care Home,7,0,7,7,7,7
Combined License,114,0,114,114,114,114


In [55]:
df[df['LICENSE_TYPE'] == 'Retail Food Establishment'].groupby('LICENSE_SUB_TYPE').count()

,BFN,LICENSE_TYPE,LICENSE_STATUS,EXPIRATION_DATE,ENTITY_NAME,ESTABLISHMENT_ADDRESS
LICENSE_SUB_TYPE,,,,,,
Mobile,32,32,32,32,32,32
Restaurant,83,83,83,83,83,83
Restaurant - Stand-Up,25,25,25,25,25,25
Restaurant - Temporary,42,42,42,42,41,42


In [87]:
restaurants_df = df[(df['LICENSE_TYPE'] == 'Retail Food Establishment') & (df['LICENSE_SUB_TYPE'] == 'Restaurant')]

In [81]:
restaurants_df.head()

,BFN,LICENSE_TYPE,LICENSE_SUB_TYPE,LICENSE_STATUS,EXPIRATION_DATE,ENTITY_NAME,ESTABLISHMENT_ADDRESS
1,2021-BFN-0002028,Retail Food Establishment,Restaurant,Pending,2021-04-06 13:23:48,PETUNIA BAKE SHOP LLC,3000 W 23rd Ave Denver 80211
4,2019-BFN-0011393,Retail Food Establishment,Restaurant,Pending,2019-12-09 08:20:12,RAISING CANES RESTAURANTS LLC,1190 S Colorado Blvd Denver 80246
13,2020-BFN-0003219,Retail Food Establishment,Restaurant,Pending,2020-06-24 13:34:06,EPIQ PIZZA LLC,1147 N Broadway 103 Denver 80203
24,2020-BFN-0002411,Retail Food Establishment,Restaurant,Pending,2020-05-07 14:57:46,SAGE DINING SERVICES INC,7701 E 1st Pl C Denver 80230
37,2020-BFN-0006879,Retail Food Establishment,Restaurant,Pending,2020-12-14 15:10:35,SODEXO AMERICA LLC,2055 E Evans Ave Denver 80210


In [82]:
restaurants_df[restaurants_df['EXPIRATION_DATE'] >= "2020"]

,BFN,LICENSE_TYPE,LICENSE_SUB_TYPE,LICENSE_STATUS,EXPIRATION_DATE,ENTITY_NAME,ESTABLISHMENT_ADDRESS
1,2021-BFN-0002028,Retail Food Establishment,Restaurant,Pending,2021-04-06 13:23:48,PETUNIA BAKE SHOP LLC,3000 W 23rd Ave Denver 80211
13,2020-BFN-0003219,Retail Food Establishment,Restaurant,Pending,2020-06-24 13:34:06,EPIQ PIZZA LLC,1147 N Broadway 103 Denver 80203
24,2020-BFN-0002411,Retail Food Establishment,Restaurant,Pending,2020-05-07 14:57:46,SAGE DINING SERVICES INC,7701 E 1st Pl C Denver 80230
37,2020-BFN-0006879,Retail Food Establishment,Restaurant,Pending,2020-12-14 15:10:35,SODEXO AMERICA LLC,2055 E Evans Ave Denver 80210
53,2021-BFN-0001470,Retail Food Establishment,Restaurant,Pending,2021-03-10 11:32:24,THE FIT FOODS LLC,201 N University Blvd 108 Denver 80206
...,...,...,...,...,...,...,...
628,2021-BFN-0002051,Retail Food Establishment,Restaurant,Pending,2021-04-07 10:43:23,ERE BUCHTEL FG LLC,2023 S Colorado Blvd 103 Denver 80222
652,2020-BFN-0006128,Retail Food Establishment,Restaurant,Pending,2020-11-19 10:46:29,STARBUCKS CORPORATION,6691 N Tower Rd Denver 80249
656,2020-BFN-0006881,Retail Food Establishment,Restaurant,Pending,2020-12-14 15:15:18,SODEXO AMERICA LLC,2055 E Evans Ave Denver 80210
662,2021-BFN-0000237,Retail Food Establishment,Restaurant,Pending,2021-01-12 13:51:45,WHOLE SOL RINO LLC,3605 Walnut St Denver 80205


In [86]:
def get_zipcode(address):
    if address[-5:] == 'nver ':
        return np.NaN
    return address[-5:]

def get_city(address):
    return address[-12:-6]

def get_streetaddr(address):
    return address[:-12].strip()

In [88]:
restaurants_df['STREET_ADDRESS'] = restaurants_df['ESTABLISHMENT_ADDRESS'].map(get_streetaddr)
restaurants_df['CITY'] = 'Denver'
restaurants_df['STATE'] = 'CO'
restaurants_df['ZIPCODE'] = restaurants_df['ESTABLISHMENT_ADDRESS'].map(get_zipcode)

<ipython-input-88-576ff464961e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurants_df['STREET_ADDRESS'] = restaurants_df['ESTABLISHMENT_ADDRESS'].map(get_streetaddr)
<ipython-input-88-576ff464961e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurants_df['CITY'] = 'Denver'
<ipython-input-88-576ff464961e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,BFN,LICENSE_TYPE,LICENSE_SUB_TYPE,LICENSE_STATUS,EXPIRATION_DATE,ENTITY_NAME,ESTABLISHMENT_ADDRESS,STREET_ADDRESS,CITY,STATE,ZIPCODE
1,2021-BFN-0002028,Retail Food Establishment,Restaurant,Pending,2021-04-06 13:23:48,PETUNIA BAKE SHOP LLC,3000 W 23rd Ave Denver 80211,3000 W 23rd Ave,Denver,CO,80211
4,2019-BFN-0011393,Retail Food Establishment,Restaurant,Pending,2019-12-09 08:20:12,RAISING CANES RESTAURANTS LLC,1190 S Colorado Blvd Denver 80246,1190 S Colorado Blvd,Denver,CO,80246
13,2020-BFN-0003219,Retail Food Establishment,Restaurant,Pending,2020-06-24 13:34:06,EPIQ PIZZA LLC,1147 N Broadway 103 Denver 80203,1147 N Broadway 103,Denver,CO,80203
24,2020-BFN-0002411,Retail Food Establishment,Restaurant,Pending,2020-05-07 14:57:46,SAGE DINING SERVICES INC,7701 E 1st Pl C Denver 80230,7701 E 1st Pl C,Denver,CO,80230
37,2020-BFN-0006879,Retail Food Establishment,Restaurant,Pending,2020-12-14 15:10:35,SODEXO AMERICA LLC,2055 E Evans Ave Denver 80210,2055 E Evans Ave,Denver,CO,80210


In [109]:
restaurants_df.head()

,BFN,LICENSE_TYPE,LICENSE_SUB_TYPE,LICENSE_STATUS,EXPIRATION_DATE,ENTITY_NAME,ESTABLISHMENT_ADDRESS,STREET_ADDRESS,CITY,STATE,ZIPCODE
1,2021-BFN-0002028,Retail Food Establishment,Restaurant,Pending,2021-04-06 13:23:48,PETUNIA BAKE SHOP LLC,3000 W 23rd Ave Denver 80211,3000 W 23rd Ave,Denver,CO,80211
4,2019-BFN-0011393,Retail Food Establishment,Restaurant,Pending,2019-12-09 08:20:12,RAISING CANES RESTAURANTS LLC,1190 S Colorado Blvd Denver 80246,1190 S Colorado Blvd,Denver,CO,80246
13,2020-BFN-0003219,Retail Food Establishment,Restaurant,Pending,2020-06-24 13:34:06,EPIQ PIZZA LLC,1147 N Broadway 103 Denver 80203,1147 N Broadway 103,Denver,CO,80203
24,2020-BFN-0002411,Retail Food Establishment,Restaurant,Pending,2020-05-07 14:57:46,SAGE DINING SERVICES INC,7701 E 1st Pl C Denver 80230,7701 E 1st Pl C,Denver,CO,80230
37,2020-BFN-0006879,Retail Food Establishment,Restaurant,Pending,2020-12-14 15:10:35,SODEXO AMERICA LLC,2055 E Evans Ave Denver 80210,2055 E Evans Ave,Denver,CO,80210


In [89]:
restaurants_df[['STREET_ADDRESS', 'CITY', 'STATE', 'ZIPCODE']].to_csv('restdf_addr_tocoord.csv', header=None)

In [125]:
new_df = pd.read_csv('GeocodeResults.csv', names=['index', 'given_address', 'match', 'match_type', 'given_input', 'coords', 'unk_val', 'unk_val2'], header=None)
new_df.head()

,index,given_address,match,match_type,given_input,coords,unk_val,unk_val2
0,190,"1195 S Colorado Blvd, Denver, CO, 80246",Match,Non_Exact,"1195 S COLORADO BLVD, DENVER, CO, 80246","-104.94074,39.69493",639438114.0,R
1,191,"3743 N Federal Blvd, Denver, CO, 80211",Match,Exact,"3743 FEDERAL BLVD, DENVER, CO, 80211","-105.02525,39.768547",177300887.0,L
2,192,"1300 S Broadway, Denver, CO, 80210",Match,Non_Exact,"1300 S BROADWAY, DENVER, CO, 80210","-104.987465,39.69287",177331283.0,L
3,150,"7357 E 29th Ave, Denver, CO, 80238",Match,Non_Exact,"7357 E 29TH AVE, DENVER, CO, 80238","-104.90283,39.758022",606059037.0,L
4,470,"2449 Larimer St, Denver, CO, 80205",Match,Exact,"2449 LARIMER ST, DENVER, CO, 80205","-104.98705,39.757244",177303976.0,L


In [126]:
new_df = new_df.set_index('index')
new_df.head()

,given_address,match,match_type,given_input,coords,unk_val,unk_val2
index,,,,,,,
190,"1195 S Colorado Blvd, Denver, CO, 80246",Match,Non_Exact,"1195 S COLORADO BLVD, DENVER, CO, 80246","-104.94074,39.69493",639438114.0,R
191,"3743 N Federal Blvd, Denver, CO, 80211",Match,Exact,"3743 FEDERAL BLVD, DENVER, CO, 80211","-105.02525,39.768547",177300887.0,L
192,"1300 S Broadway, Denver, CO, 80210",Match,Non_Exact,"1300 S BROADWAY, DENVER, CO, 80210","-104.987465,39.69287",177331283.0,L
150,"7357 E 29th Ave, Denver, CO, 80238",Match,Non_Exact,"7357 E 29TH AVE, DENVER, CO, 80238","-104.90283,39.758022",606059037.0,L
470,"2449 Larimer St, Denver, CO, 80205",Match,Exact,"2449 LARIMER ST, DENVER, CO, 80205","-104.98705,39.757244",177303976.0,L


In [127]:
new_df.sort_index()

,given_address,match,match_type,given_input,coords,unk_val,unk_val2
index,,,,,,,
1,"3000 W 23rd Ave, Denver, CO, 80211",Match,Non_Exact,"3000 W 23RD AVE, DENVER, CO, 80211","-105.02535,39.7512",177301496.0,L
4,"1190 S Colorado Blvd, Denver, CO, 80246",Match,Non_Exact,"1190 S COLORADO BLVD, DENVER, CO, 80246","-104.94062,39.69459",612641847.0,L
13,"1147 N Broadway 103, Denver, CO, 80203",Match,Exact,"1147 BROADWAY, DENVER, CO, 80203","-104.98741,39.73453",177304788.0,L
24,"7701 E 1st Pl C, Denver, CO, 80230",Match,Non_Exact,"7701 E 1ST PL, DENVER, CO, 80230","-104.89654,39.72211",640072451.0,R
37,"2055 E Evans Ave, Denver, CO, 80210",Match,Non_Exact,"2055 E EVANS AVE, DENVER, CO, 80210","-104.96343,39.67851",177623972.0,L
...,...,...,...,...,...,...,...
628,"2023 S Colorado Blvd 103, Denver, CO, 80222",Match,Non_Exact,"2023 S COLORADO BLVD, DENVER, CO, 80222","-104.94087,39.679634",639438177.0,R
652,"6691 N Tower Rd, Denver, CO, 80249",Match,Exact,"6691 TOWER RD, DENVER, CO, 80249","-104.77203,39.817997",650196118.0,L
656,"2055 E Evans Ave, Denver, CO, 80210",Match,Non_Exact,"2055 E EVANS AVE, DENVER, CO, 80210","-104.96343,39.67851",177623972.0,L


In [128]:
new_df[new_df.coords.isnull()]

,given_address,match,match_type,given_input,coords,unk_val,unk_val2
index,,,,,,,


In [129]:
new_df = pd.concat([new_df, new_df['coords'].str.split(',', expand=True)], axis=1)
new_df.head()

,given_address,match,match_type,given_input,coords,unk_val,unk_val2,0,1
index,,,,,,,,,
190,"1195 S Colorado Blvd, Denver, CO, 80246",Match,Non_Exact,"1195 S COLORADO BLVD, DENVER, CO, 80246","-104.94074,39.69493",639438114.0,R,-104.94074,39.69493
191,"3743 N Federal Blvd, Denver, CO, 80211",Match,Exact,"3743 FEDERAL BLVD, DENVER, CO, 80211","-105.02525,39.768547",177300887.0,L,-105.02525,39.768547
192,"1300 S Broadway, Denver, CO, 80210",Match,Non_Exact,"1300 S BROADWAY, DENVER, CO, 80210","-104.987465,39.69287",177331283.0,L,-104.987465,39.69287
150,"7357 E 29th Ave, Denver, CO, 80238",Match,Non_Exact,"7357 E 29TH AVE, DENVER, CO, 80238","-104.90283,39.758022",606059037.0,L,-104.90283,39.758022
470,"2449 Larimer St, Denver, CO, 80205",Match,Exact,"2449 LARIMER ST, DENVER, CO, 80205","-104.98705,39.757244",177303976.0,L,-104.98705,39.757244


In [142]:
new_df.rename(columns={0:'longitude', 1:'latitude'}, inplace=True)

In [144]:
new_df

,given_address,match,match_type,given_input,coords,unk_val,unk_val2,longitude,latitude
index,,,,,,,,,
190,"1195 S Colorado Blvd, Denver, CO, 80246",Match,Non_Exact,"1195 S COLORADO BLVD, DENVER, CO, 80246","-104.94074,39.69493",639438114.0,R,-104.94074,39.69493
191,"3743 N Federal Blvd, Denver, CO, 80211",Match,Exact,"3743 FEDERAL BLVD, DENVER, CO, 80211","-105.02525,39.768547",177300887.0,L,-105.02525,39.768547
192,"1300 S Broadway, Denver, CO, 80210",Match,Non_Exact,"1300 S BROADWAY, DENVER, CO, 80210","-104.987465,39.69287",177331283.0,L,-104.987465,39.69287
150,"7357 E 29th Ave, Denver, CO, 80238",Match,Non_Exact,"7357 E 29TH AVE, DENVER, CO, 80238","-104.90283,39.758022",606059037.0,L,-104.90283,39.758022
470,"2449 Larimer St, Denver, CO, 80205",Match,Exact,"2449 LARIMER ST, DENVER, CO, 80205","-104.98705,39.757244",177303976.0,L,-104.98705,39.757244
...,...,...,...,...,...,...,...,...,...
547,"3550 W 38th Ave 80, Denver, CO, 80211",Match,Non_Exact,"3550 W 38TH AVE, DENVER, CO, 80211","-105.03407,39.76952",177300655.0,L,-105.03407,39.76952
306,"1998 N Downing St, Denver, CO, 80218",No_Match,NaN,NaN,"-104.97281995941235,39.74668421718216",NaN,NaN,-104.97281995941235,39.74668421718216
309,"24 N Broadway, Denver, CO, 80203",Match,Exact,"24 BROADWAY, DENVER, CO, 80203","-104.987495,39.716835",177330871.0,R,-104.987495,39.716835
